In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Preprocessing***

In [1]:
import pandas as pd
import numpy as np
# Plot histograms for sentiment_score and Close price
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.stattools import grangercausalitytests
from collections import Counter
import re
from scipy.stats import pearsonr
from collections import Counter
import nltk
from nltk.corpus import stopwords
import gensim
from gensim import corpora

In [2]:
filepath_numerical = '/content/drive/My Drive/nlp/dataset_final/finsen_s&p500/S&P500_numerical.csv'
filepath_sentiment = '/content/drive/My Drive/nlp/dataset_final/finsen_s&p500/FinSen_text_annotated.csv'

In [3]:
data_num = pd.read_csv(filepath_numerical)
data_num.head()

,Price,Open,Close,High,Volume
0,Ticker,^GSPC,^GSPC,^GSPC,^GSPC
1,Date,NaN,NaN,NaN,NaN
2,2023-07-14,4514.6,4505.41,4527.75,3647450000.0
3,2023-07-13,4491.5,4510.04,4517.37,3839530000.0
4,2023-07-12,4467.68,4472.16,4488.33,3920290000.0


In [4]:
data_sent = pd.read_csv(filepath_sentiment)
data_sent.head()

,Unnamed: 0,Title,Tag,Time,Content,sentiment_score
0,0,"TSX Slightly Down, Books Weekly Gains",Stock Market,16/07/2023,"TSX Slightly Down, Books Weekly GainsUnited St...",0.13
1,1,UnitedHealth Hits 4-week High,stocks,15/07/2023,UnitedHealth Hits 4-week HighUnited States sto...,0.62
2,2,Cisco Systems Hits 4-week Low,stocks,15/07/2023,Cisco Systems Hits 4-week LowUnited States sto...,-0.43
3,3,AT&T Hits All-time Low,stocks,15/07/2023,AT&T Hits All-time LowUnited States stocksAT&T...,-0.28
4,4,Microsoft Hits 4-week High,stocks,15/07/2023,Microsoft Hits 4-week HighUnited States stocks...,0.60


In [5]:
if 'Unnamed: 0' in data_sent.columns:
    data_sent = data_sent.drop(columns=['Unnamed: 0'])
data_sent.rename(columns={'Time': 'Date'}, inplace=True)
data_sent.head()

,Title,Tag,Date,Content,sentiment_score
0,"TSX Slightly Down, Books Weekly Gains",Stock Market,16/07/2023,"TSX Slightly Down, Books Weekly GainsUnited St...",0.13
1,UnitedHealth Hits 4-week High,stocks,15/07/2023,UnitedHealth Hits 4-week HighUnited States sto...,0.62
2,Cisco Systems Hits 4-week Low,stocks,15/07/2023,Cisco Systems Hits 4-week LowUnited States sto...,-0.43
3,AT&T Hits All-time Low,stocks,15/07/2023,AT&T Hits All-time LowUnited States stocksAT&T...,-0.28
4,Microsoft Hits 4-week High,stocks,15/07/2023,Microsoft Hits 4-week HighUnited States stocks...,0.60


In [6]:
data_sent.value_counts()

,,,,,count
Title,Tag,Date,Content,sentiment_score,
US Core PPI Rises Less than Expected,Core Producer Prices MoM,14/07/2023,"US Core PPI Rises Less than ExpectedUnited States Core Producer Prices MoMCore producer prices in the United States increased by 0.1% over a month in June 2023, matching May's revised reading and slightly below market estimates of a 0.2% rise. On a yearly basis, core consumer prices advanced by 2.4% in June, slowing from a downwardly revised 2.6% increase in May and below market expectations of a 2.6% rise.",0.00,2
Dollar Languishes on Dovish Fed Bets,Currency,15/07/2023,"Dollar Languishes on Dovish Fed BetsUnited States CurrencyThe dollar index held its recent decline to below 100 on Friday, sinking to its lowest levels in 15 months as signs that US inflation is on a downward trend raised hopes that the Federal Reserve is nearing the end of its current monetary policy tightening cycle. Latest data showed that US consumer and producer inflation rose less than anticipated in June even as the US labor market remained robust. The US central bank is still seen raising rates by 25 basis points in July, while traders scaled back bets of further rate increases after this year. Market pricing also suggests that the Fed could start cutting rates next year. The dollar plunged to over one-year lows against the euro and sterling, while it declined to multi-month lows against the yen and antipodean currencies.",-0.27,2
Progressive earnings below expectations at 0.57 USD,Earnings,14/07/2023,"Progressive earnings below expectations at 0.57 USDUnited States EarningsProgressive (PGR) released earnings per share at 0.57 USD, compared to market expectations of 0.89 USD.",0.01,2
US Budget Deficit Widens More than Expected in June,Government Budget Value,14/07/2023,"US Budget Deficit Widens More than Expected in JuneUnited States Government Budget ValueThe US government budget deficit came at USD 227.8 billion in June 2023, widening from a USD 88.8 billion gap in the same month of the previous year, and below market expectations of a USD 175 billion deficit as revenues continued to weaken and July benefit payments were accelerated into June. Revenues declined by 9.2% to USD 418 billion due to lower non-withheld individual income taxes. Outlays soared 17.5% to $646 billion led by higher individual tax refunds as the Internal Revenue Service cleared a backlog of unprocessed receipts. Considering the first nine months of the fiscal year, the budget deficit stood at USD 1.393 trillion compared to a USD 1.375 billion deficit a year earlier.",-0.06,2
PepsiCo earnings above expectations at 2.09 USD,Earnings,14/07/2023,"PepsiCo earnings above expectations at 2.09 USDUnited States EarningsPepsiCo (PEP) released earnings per share at 2.09 USD, compared to market expectations of 1.96 USD.",0.01,2
...,...,...,...,...,...
Philadelphia Factory Activity Unexpectedly Grows,Philadelphia Fed Manufacturing Index,17/03/2022,"Philadelphia Factory Activity Unexpectedly GrowsUnited States Philadelphia Fed Manufacturing IndexThe Philadelphia Fed Manufacturing Index in the US rose to 27.4 in March of 2022 from 16 in February and above market expectations of 15. It was the highest reading since last November. The survey’s indicators for general activity, shipments, and new orders all rose after declining last month. The employment index and both price indexes climbed higher and remain elevated. The survey’s future general activity, new orders, and shipments indexes moderated, but the surveyed firms remained generally optimistic about growth over the next six months.",0.32,1
Philadelphia Factory Growth Eases for 2nd Month,Philadelphia Fed Manufacturing Index,17/06/2021,"Philadelphia Factory Growth Eases for 2nd MonthUnited States Philadelphia Fed Manufacturing IndexThe Philadelphia Fed Manufacturing Index in the US fell for the second month to 30.7 in June of 2021 from 31.5 in May, compared to market forecasts of 31. Still, the reading showed manufact

In [7]:
data_sent = data_sent.drop_duplicates(subset=['Title', 'Content'])
data_sent.value_counts()

,,,,,count
Title,Tag,Date,Content,sentiment_score,
10-Year Treasury Note Approaches 3-Month Low After CPI,Government Bond 10Y,13/12/2022,"10-Year Treasury Note Approaches 3-Month Low After CPIUnited States Government Bond 10YThe yield on the US 10-year Treasury noted retreated below the 3.5% mark, approaching the three-month low of 3.4% touched on December 7th after the domestic inflation report surprised to the downside, easing expectations of how high the Federal Reserve will have to raise its funds rate target. Consumer prices rose by 7.1% annually in November, below estimates of 7.3% and easing from 7.7% in the prior month. The result consolidated hopes of slowing price growth as data showed that consumer inflation expectations dropped to the lowest in 14 months, adding to bets that the Federal Reserve will deliver a 50bps rate hike this week and maintain smaller rate hike increments after four straight 75bps increases since June.",-0.01,1
US Industrial Output Rises The Most In 3 Years,Industrial Production Mom,16/05/2017,"US Industrial Output Rises The Most In 3 YearsUnited States Industrial Production MomUS industrial production increased by 1 percent month-over-month in April 2017, following a downwardly revised 0.4 percent rise in March and beating market expectations of a 0.3 percent gain. It was the largest increase in industrial production since February 2014, as output rebounded for both manufacturing and mining while utilities grew at a slower pace.",0.23,1
US Industrial Production Growth Hits 5-Month High,Industrial Production,18/10/2022,"US Industrial Production Growth Hits 5-Month HighUnited States Industrial ProductionIndustrial production in the United States increased 5.3 percent year-on-year in September 2022, the most since April, accelerating from an upwardly revised 3.9 percent advance in the previous month. Manufacturing output growth accelerated to 4.7 percent from 3.5 percent in August, due to a 6.0 percent jump in the production of durable manufacturing, led by motor vehicles and parts, fabricated metal products, machinery, and aerospace and miscellaneous transportation equipment. Also, nondurable manufacturing increased 3.6 percent, boosted by the production of food, beverage, and tobacco products, and chemicals. Mining output jumped 11.1 percent (8.7 percent), while utilities output moved up 0.5 percent as a 4.3 percent advance in natural gas production was partially offset by a 0.3 percent decline in electric output. On a monthly basis, industrial output rose 0.4 percent in September, beating market forecasts of 0.1 percent.",0.50,1
US Industrial Production Growth Strongest in 8-Months,Industrial Production Mom,14/08/2015,US Industrial Production Growth Strongest in 8-MonthsUnited States Industrial Production MomIndustrial output in the US rose 0.6 percent in July after a downwardly revised increase of 0.1 percent in June. The gain was a result of a 0.8 percent growth in factory production supported by a surge in production of motor vehicles.,0.22,1
US Industrial Production Rebounds,Industrial Production Mom,15/07/2015,"US Industrial Production ReboundsUnited States Industrial Production MomIndustrial output rose 0.3 percent in June from May, beating market expectations and recovering from contraction in the previous two months. It is the highest gain in seven months boosted by mining and utilities while manufacturing was flat.",0.46,1
...,...,...,...,...,...
Philadelphia Fed Manufacturing Index At 2-Year High,Philadelphia Fed Manufacturing Index,15/12/2016,"Philadelphia Fed Manufacturing Index At 2-Year High United States Philadelphia Fed Manufacturing IndexThe Philadelphia Fed Manufacturing Index surged to 21.5 in December of 2016 from 7.6 in November and well above market expectations of 9. It was the highest reading since November 2014. The shipments increased to 22 (from 19.5) and employment rebounded (6.4 from -2.6). Meantime, new orders index went down to 13.9 (from 18.6) and inventories de

***Merge Dataframes***

In [8]:
# Aggregate sentiment scores by summing them up for each date
data_sent_aggregated = (
    data_sent.groupby('Date', as_index=False)
    .agg({'sentiment_score': 'sum'})
)

# Round the sentiment_score to two decimal places
data_sent_aggregated['sentiment_score'] = data_sent_aggregated['sentiment_score'].round(2)

# Merge aggregated scores back into the original DataFrame
data_sent = data_sent.drop(columns=['sentiment_score'])  # Drop original sentiment_score column
data_sent = data_sent.merge(data_sent_aggregated, on='Date', how='left')  # Add aggregated column

print(data_sent)


                                                   Title               Tag  \
0                  TSX Slightly Down, Books Weekly Gains      Stock Market   
1                          UnitedHealth Hits 4-week High            stocks   
2                          Cisco Systems Hits 4-week Low            stocks   
3                                 AT&T Hits All-time Low            stocks   
4                             Microsoft Hits 4-week High            stocks   
...                                                  ...               ...   
15478  United States GDP Rises 0.6 percent in the fir...   GDP Growth Rate   
15479  Consumer Price Index 2.6 percent higher than i...    Inflation Rate   
15480  U.S. Federal Reserve Kept Rates Unchanged at 5...     Interest Rate   
15481              Trade Deficit Increases in March 2007  Balance of Trade   
15482  Blackstone boosts IPO after Beijing takes $3bn...              News   

             Date                                            Co

In [11]:
import pandas as pd

# --- Step 1: Clean the data_num DataFrame ---

# Remove the first two bad rows (index 0 and 1)
# We use .copy() to avoid a SettingWithCopyWarning
data_num_cleaned = data_num.iloc[2:].copy()

# Rename the 'Price' column to 'Date' since it contains the dates
data_num_cleaned.rename(columns={'Price': 'Date'}, inplace=True)


# --- Step 2: Convert Date columns to datetime format ---

# Convert the new 'Date' column in the cleaned dataframe
data_num_cleaned['Date'] = pd.to_datetime(data_num_cleaned['Date'])

# Convert the 'Date' column in data_sent.
# Use dayfirst=True because your format is DD/MM/YYYY
data_sent['Date'] = pd.to_datetime(data_sent['Date'], dayfirst=True)


# --- Step 3: Merge the cleaned dataframes ---

# Merge on the 'Date' column
data_merged = pd.merge(data_num_cleaned, data_sent, on='Date', how='inner')

# --- Step 4: Verify the result ---
print("Merge successful! Here is the head of the merged data:")
print(data_merged.head())

Merge successful! Here is the head of the merged data:
        Date    Open    Close     High        Volume  \
0 2023-07-14  4514.6  4505.41  4527.75  3647450000.0   
1 2023-07-14  4514.6  4505.41  4527.75  3647450000.0   
2 2023-07-14  4514.6  4505.41  4527.75  3647450000.0   
3 2023-07-14  4514.6  4505.41  4527.75  3647450000.0   
4 2023-07-14  4514.6  4505.41  4527.75  3647450000.0   

                                               Title                      Tag  \
0  US Budget Deficit Widens More than Expected in...  Government Budget Value   
1                             Visa Hits 24-week High                   stocks   
2                           Amazon Hits 43-week High                   stocks   
3       10-Year Treasury Yield Falls for 4th Session      Government Bond 10Y   
4                                 DXY Approaches 100                 Currency   

                                             Content  sentiment_score  
0  US Budget Deficit Widens More than Expected in

In [12]:
data_merged.head()

,Date,Open,Close,High,Volume,Title,Tag,Content,sentiment_score
0,2023-07-14,4514.6,4505.41,4527.75,3647450000.0,US Budget Deficit Widens More than Expected in...,Government Budget Value,US Budget Deficit Widens More than Expected in...,1.02
1,2023-07-14,4514.6,4505.41,4527.75,3647450000.0,Visa Hits 24-week High,stocks,Visa Hits 24-week HighUnited States stocksVisa...,1.02
2,2023-07-14,4514.6,4505.41,4527.75,3647450000.0,Amazon Hits 43-week High,stocks,Amazon Hits 43-week HighUnited States stocksAm...,1.02
3,2023-07-14,4514.6,4505.41,4527.75,3647450000.0,10-Year Treasury Yield Falls for 4th Session,Government Bond 10Y,10-Year Treasury Yield Falls for 4th SessionUn...,1.02
4,2023-07-14,4514.6,4505.41,4527.75,3647450000.0,DXY Approaches 100,Currency,DXY Approaches 100United States CurrencyThe do...,1.02


In [13]:
# Ensure filtered_data is a full copy of data_merged
filtered_data = data_merged.copy()

# Merge all rows for the same date into one row
merged_data = filtered_data.groupby('Date', as_index=False).agg({
    'Open': 'first',
    'Close': 'first',
    'High': 'first',
    'Volume': 'sum',  # Summing volumes for the same date
    'Title': lambda x: ', '.join(x),  # Joining titles with commas
    'Tag': lambda x: ', '.join(x),  # Joining tags with commas
    'Content': lambda x: ', '.join(x),  # Joining content with commas
    'sentiment_score': 'first'  # Assuming sentiment score is already aggregated
})

# Display or save the result
merged_data.head(10)


,Date,Open,Close,High,Volume,Title,Tag,Content,sentiment_score
0,2007-06-04,1536.28,1539.18,1540.53,2738930000.02738930000.0,"Trade Deficit Increases in March 2007, Blackst...","Balance of Trade, News",Trade Deficit Increases in March 2007United St...,0.03
1,2007-06-06,1530.56,1517.38,1530.56,2964190000.02964190000.02964190000.02964190000...,"Nonfarm payroll employment increased by 157,00...","Unemployment Rate, Balance of Trade, GDP Growt...","Nonfarm payroll employment increased by 157,00...",0.01
2,2007-06-21,1512.5,1522.18,1522.9,3161110000.0,US Economy is expanding after Q1 slowdown,GDP Growth Rate,US Economy is expanding after Q1 slowdownUnite...,0.03
3,2007-06-26,1497.68,1492.89,1506.11,3398530000.0,Consumer Confidence and Housing Weaken in US,GDP Growth Rate,Consumer Confidence and Housing Weaken in USUn...,-0.66
4,2007-06-28,1506.31,1505.7,1514.83,3006710000.0,Fed keeps the federal funds rate at 5-1/4 percent,Interest Rate,Fed keeps the federal funds rate at 5-1/4 perc...,0.16
5,2007-07-05,1524.85,1525.4,1526.56,2622950000.0,U.S. ISM Services Index Reached 14-Month High ...,News,U.S. ISM Services Index Reached 14-Month High ...,0.66
6,2007-07-06,1524.95,1530.43,1532.4,2441520000.0,US June unemployment rate unchanged at 4.5%,Unemployment Rate,US June unemployment rate unchanged at 4.5%Uni...,-0.02
7,2007-07-10,1531.84,1510.11,1531.84,3244280000.0,"U.S. Growth, Easing Inflation to Follow Fed `S...",Inflation Rate,"U.S. Growth, Easing Inflation to Follow Fed `S...",0.00
8,2007-07-12,1518.73,1547.69,1547.92,3489600000.0,US trade deficit increases to $60.0 billion in...,Balance of Trade,US trade deficit increases to $60.0 billion in...,-0.02
9,2007-07-18,1549.19,1546.17,1549.19,3609220000.0,Fed chief acknowledges credit fears,Stock Market,Fed chief acknowledges credit fearsUnited Stat...,-0.11


***Movement calculation - Open (t+1) - Close (t)***

In [21]:
import pandas as pd

# --- FIX: Convert columns to a numeric type ---
# Use errors='coerce' to turn any non-numeric values into NaN
merged_data['Open'] = pd.to_numeric(merged_data['Open'], errors='coerce')
merged_data['Close'] = pd.to_numeric(merged_data['Close'], errors='coerce')

# Sort the index (if it's not already sorted)
merged_data = merged_data.sort_index()

# Calculate next-day return (this will now work)
merged_data['Movement'] = (merged_data['Open'].shift(-1) - merged_data['Close']) / merged_data['Close']

# Drop the last row and any rows that failed conversion
merged_data = merged_data.dropna(subset=['Movement'])

# Display the result
print(merged_data.head())

        Date     Open    Close     High        Volume  \
2 2007-06-21  1512.50  1522.18  1522.90  3.161110e+09   
3 2007-06-26  1497.68  1492.89  1506.11  3.398530e+09   
4 2007-06-28  1506.31  1505.70  1514.83  3.006710e+09   
5 2007-07-05  1524.85  1525.40  1526.56  2.622950e+09   
6 2007-07-06  1524.95  1530.43  1532.40  2.441520e+09   

                                               Title                Tag  \
2          US Economy is expanding after Q1 slowdown    GDP Growth Rate   
3       Consumer Confidence and Housing Weaken in US    GDP Growth Rate   
4  Fed keeps the federal funds rate at 5-1/4 percent      Interest Rate   
5  U.S. ISM Services Index Reached 14-Month High ...               News   
6        US June unemployment rate unchanged at 4.5%  Unemployment Rate   

                                             Content  sentiment_score  \
2  US Economy is expanding after Q1 slowdownUnite...             0.03   
3  Consumer Confidence and Housing Weaken in USUn...        

In [22]:
merged_data.head()

,Date,Open,Close,High,Volume,Title,Tag,Content,sentiment_score,Movement
2,2007-06-21,1512.50,1522.18,1522.90,3.161110e+09,US Economy is expanding after Q1 slowdown,GDP Growth Rate,US Economy is expanding after Q1 slowdownUnite...,0.03,-0.016095
3,2007-06-26,1497.68,1492.89,1506.11,3.398530e+09,Consumer Confidence and Housing Weaken in US,GDP Growth Rate,Consumer Confidence and Housing Weaken in USUn...,-0.66,0.008989
4,2007-06-28,1506.31,1505.70,1514.83,3.006710e+09,Fed keeps the federal funds rate at 5-1/4 percent,Interest Rate,Fed keeps the federal funds rate at 5-1/4 perc...,0.16,0.012718
5,2007-07-05,1524.85,1525.40,1526.56,2.622950e+09,U.S. ISM Services Index Reached 14-Month High ...,News,U.S. ISM Services Index Reached 14-Month High ...,0.66,-0.000295
6,2007-07-06,1524.95,1530.43,1532.40,2.441520e+09,US June unemployment rate unchanged at 4.5%,Unemployment Rate,US June unemployment rate unchanged at 4.5%Uni...,-0.02,0.000921


In [23]:
# Assuming df already has Next_Day_Return and is cleaned
# List of features you want to analyze
feature_cols = ['Open', 'Close', 'High', 'Volume', 'sentiment_score']

# Calculate the correlation matrix for all features including Next_Day_Return
corr_matrix = merged_data[feature_cols + ['Movement']].corr()

# Print the correlation of each feature with Next_Day_Return
print("Correlation with Next_Day_Return:")
for col in feature_cols:
    print(f"{col}: {corr_matrix.loc[col, 'Movement']:.4f}")


Correlation with Next_Day_Return:
Open: 0.1410
Close: 0.1370
High: 0.1392
Volume: -0.0627
sentiment_score: 0.0084


***Feature Engineering***

***1. Lagged Features***

In [24]:
# Create lagged features (e.g., 1-day lag)
merged_data['Open_lag1'] = merged_data['Open'].shift(1)
merged_data['Close_lag1'] = merged_data['Close'].shift(1)
merged_data['High_lag1'] = merged_data['High'].shift(1)
merged_data['Volume_lag1'] = merged_data['Volume'].shift(1)
merged_data['Sentiment_lag1'] = merged_data['sentiment_score'].shift(1)

# Drop rows introduced by shifting at the start of dataset
merged_data = merged_data.dropna(subset=['Open_lag1', 'Close_lag1', 'High_lag1', 'Volume_lag1', 'Sentiment_lag1', 'Movement'])


In [25]:
# Create lagged features (e.g., 2-day lag)
merged_data['Open_lag2'] = merged_data['Open'].shift(2)
merged_data['Close_lag2'] = merged_data['Close'].shift(2)
merged_data['High_lag2'] = merged_data['High'].shift(2)
merged_data['Volume_lag2'] = merged_data['Volume'].shift(2)
merged_data['Sentiment_lag2'] = merged_data['sentiment_score'].shift(2)

# Drop rows introduced by shifting at the start of dataset
merged_data = merged_data.dropna(subset=['Open_lag2', 'Close_lag2', 'High_lag2', 'Volume_lag2', 'Sentiment_lag2', 'Movement'])


In [26]:
# Create lagged features (e.g., 3-day lag)
merged_data['Open_lag3'] = merged_data['Open'].shift(3)
merged_data['Close_lag3'] = merged_data['Close'].shift(3)
merged_data['High_lag3'] = merged_data['High'].shift(3)
merged_data['Volume_lag3'] = merged_data['Volume'].shift(3)
merged_data['Sentiment_lag3'] = merged_data['sentiment_score'].shift(3)

# Drop rows introduced by shifting at the start of dataset
merged_data = merged_data.dropna(subset=['Open_lag3', 'Close_lag3', 'High_lag3', 'Volume_lag3', 'Sentiment_lag3', 'Movement'])


In [27]:
# Update feature list with newly created features
new_features = ['Open_lag1', 'Close_lag1', 'High_lag1', 'Volume_lag1', 'Sentiment_lag1','Open_lag2', 'Close_lag2', 'High_lag2', 'Volume_lag2', 'Sentiment_lag2','Open_lag3', 'Close_lag3', 'High_lag3', 'Volume_lag3', 'Sentiment_lag3']

# Compute correlation matrix again
corr_matrix = merged_data[new_features + ['Movement']].corr()

print("Correlation with Movement:")
for f in new_features:
    print(f"{f}: {corr_matrix.loc[f, 'Movement']:.4f}")


Correlation with Movement:
Open_lag1: 0.1529
Close_lag1: 0.1545
High_lag1: 0.1540
Volume_lag1: -0.1102
Sentiment_lag1: -0.0022
Open_lag2: 0.1362
Close_lag2: 0.1375
High_lag2: 0.1365
Volume_lag2: -0.0655
Sentiment_lag2: -0.0335
Open_lag3: 0.1448
Close_lag3: 0.1469
High_lag3: 0.1459
Volume_lag3: -0.0936
Sentiment_lag3: 0.0501


***2. Rolling Averages***

In [28]:
# 5-day windows
merged_data['Open_5d_SMA'] = merged_data['Open'].rolling(window=5).mean()
merged_data['Close_5d_SMA'] = merged_data['Close'].rolling(window=5).mean()
merged_data['High_5d_SMA'] = merged_data['High'].rolling(window=5).mean()
merged_data['Volume_5d_SMA'] = merged_data['Volume'].rolling(window=5).mean()
merged_data['Sentiment_5d_SMA'] = merged_data['sentiment_score'].rolling(window=5).mean()

# 7-day windows
merged_data['Open_7d_SMA'] = merged_data['Open'].rolling(window=7).mean()
merged_data['Close_7d_SMA'] = merged_data['Close'].rolling(window=7).mean()
merged_data['High_7d_SMA'] = merged_data['High'].rolling(window=7).mean()
merged_data['Volume_7d_SMA'] = merged_data['Volume'].rolling(window=7).mean()
merged_data['Sentiment_7d_SMA'] = merged_data['sentiment_score'].rolling(window=7).mean()

# 7-day windows
merged_data['Open_14d_SMA'] = merged_data['Open'].rolling(window=14).mean()
merged_data['Close_14d_SMA'] = merged_data['Close'].rolling(window=14).mean()
merged_data['High_14d_SMA'] = merged_data['High'].rolling(window=14).mean()
merged_data['Volume_14d_SMA'] = merged_data['Volume'].rolling(window=14).mean()
merged_data['Sentiment_14d_SMA'] = merged_data['sentiment_score'].rolling(window=14).mean()


# Drop rows with NaN due to rolling at dataset start
merged_data = merged_data.dropna(subset=['Open_5d_SMA', 'Close_5d_SMA', 'High_5d_SMA', 'Volume_5d_SMA', 'Sentiment_5d_SMA', 'Open_7d_SMA', 'Close_7d_SMA', 'High_7d_SMA', 'Volume_7d_SMA', 'Sentiment_7d_SMA', 'Open_14d_SMA', 'Close_14d_SMA', 'High_14d_SMA', 'Volume_14d_SMA', 'Sentiment_14d_SMA', 'Movement'])


In [29]:
# Update feature list with newly created features
new_features = ['Open_5d_SMA', 'Close_5d_SMA', 'High_5d_SMA', 'Volume_5d_SMA', 'Sentiment_5d_SMA', 'Open_7d_SMA', 'Close_7d_SMA', 'High_7d_SMA', 'Volume_7d_SMA', 'Sentiment_7d_SMA', 'Open_14d_SMA', 'Close_14d_SMA', 'High_14d_SMA', 'Volume_14d_SMA', 'Sentiment_14d_SMA']

# Compute correlation matrix again
corr_matrix = merged_data[new_features + ['Movement']].corr()

print("Correlation with Movement:")
for f in new_features:
    print(f"{f}: {corr_matrix.loc[f, 'Movement']:.4f}")

Correlation with Movement:
Open_5d_SMA: 0.1435
Close_5d_SMA: 0.1439
High_5d_SMA: 0.1437
Volume_5d_SMA: -0.0886
Sentiment_5d_SMA: 0.0231
Open_7d_SMA: 0.1440
Close_7d_SMA: 0.1444
High_7d_SMA: 0.1440
Volume_7d_SMA: -0.0938
Sentiment_7d_SMA: 0.0264
Open_14d_SMA: 0.1408
Close_14d_SMA: 0.1409
High_14d_SMA: 0.1406
Volume_14d_SMA: -0.0810
Sentiment_14d_SMA: 0.0609


***3. Rolling Volatility***

In [30]:
merged_data['Open_10d_STD'] = merged_data['Open'].rolling(window=10).std()
merged_data['Close_10d_STD'] = merged_data['Close'].rolling(window=10).std()
merged_data['High_10d_STD'] = merged_data['High'].rolling(window=10).std()
merged_data['Volume_10d_STD'] = merged_data['Volume'].rolling(window=10).std()
merged_data['Sentiment_10d_STD'] = merged_data['sentiment_score'].rolling(window=10).std()

# Drop rows with NaN due to rolling
merged_data = merged_data.dropna(subset=['Open_10d_STD', 'Close_10d_STD', 'High_10d_STD', 'Volume_10d_STD', 'Sentiment_10d_STD', 'Movement'])


In [31]:
# Update feature list with newly created features
new_features = ['Open_10d_STD', 'Close_10d_STD', 'High_10d_STD', 'Volume_10d_STD', 'Sentiment_10d_STD']

# Compute correlation matrix again
corr_matrix = merged_data[new_features + ['Movement']].corr()

print("Correlation with Movement:")
for f in new_features:
    print(f"{f}: {corr_matrix.loc[f, 'Movement']:.4f}")

Correlation with Movement:
Open_10d_STD: 0.1823
Close_10d_STD: 0.1924
High_10d_STD: 0.1915
Volume_10d_STD: -0.0940
Sentiment_10d_STD: -0.0001


***Causality Testing***

***1. Stationarity Testing***

In [32]:
# Let's test stationarity on the 'Movement' series
movement_series = merged_data['Movement'].dropna()

adf_result = ts.adfuller(movement_series, autolag='AIC')

print("ADF Statistic: ", adf_result[0])
print("p-value: ", adf_result[1])
for key, value in adf_result[4].items():
    print('Critical Values:')
    print(f'   {key}, {value}')


ADF Statistic:  -12.228150939609927
p-value:  1.0687044898947963e-22
Critical Values:
   1%, -3.441406876071572
Critical Values:
   5%, -2.866418015869717
Critical Values:
   10%, -2.5693678601956718


In [33]:
# Let's test stationarity on the 'sentiment' series
movement_series = merged_data['sentiment_score'].dropna()

adf_result = ts.adfuller(movement_series, autolag='AIC')

print("ADF Statistic: ", adf_result[0])
print("p-value: ", adf_result[1])
for key, value in adf_result[4].items():
    print('Critical Values:')
    print(f'   {key}, {value}')


ADF Statistic:  -4.303255022642708
p-value:  0.0004384821362803018
Critical Values:
   1%, -3.4415584920942424
Critical Values:
   5%, -2.866484781324317
Critical Values:
   10%, -2.569403436033035


***2. Granger Causality Testing***

In [34]:
merged_data.columns

Index(['Date', 'Open', 'Close', 'High', 'Volume', 'Title', 'Tag', 'Content',
       'sentiment_score', 'Movement', 'Open_lag1', 'Close_lag1', 'High_lag1',
       'Volume_lag1', 'Sentiment_lag1', 'Open_lag2', 'Close_lag2', 'High_lag2',
       'Volume_lag2', 'Sentiment_lag2', 'Open_lag3', 'Close_lag3', 'High_lag3',
       'Volume_lag3', 'Sentiment_lag3', 'Open_5d_SMA', 'Close_5d_SMA',
       'High_5d_SMA', 'Volume_5d_SMA', 'Sentiment_5d_SMA', 'Open_7d_SMA',
       'Close_7d_SMA', 'High_7d_SMA', 'Volume_7d_SMA', 'Sentiment_7d_SMA',
       'Open_14d_SMA', 'Close_14d_SMA', 'High_14d_SMA', 'Volume_14d_SMA',
       'Sentiment_14d_SMA', 'Open_10d_STD', 'Close_10d_STD', 'High_10d_STD',
       'Volume_10d_STD', 'Sentiment_10d_STD'],
      dtype='object')

In [35]:
# Drop NA values
merged_data = merged_data.dropna(subset=['sentiment_score', 'Movement'])

# Prepare data: [Affected_Series (Movement), Causing_Series (sentiment_score)]
test_data = merged_data[['Movement', 'sentiment_score']]

# Perform Granger causality test for lags from 1 to 3
max_lag = 3
granger_results = grangercausalitytests(test_data, maxlag=max_lag, verbose=True)



Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0007  , p=0.9783  , df_denom=593, df_num=1
ssr based chi2 test:   chi2=0.0007  , p=0.9783  , df=1
likelihood ratio test: chi2=0.0007  , p=0.9783  , df=1
parameter F test:         F=0.0007  , p=0.9783  , df_denom=593, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3971  , p=0.6724  , df_denom=590, df_num=2
ssr based chi2 test:   chi2=0.8010  , p=0.6700  , df=2
likelihood ratio test: chi2=0.8005  , p=0.6702  , df=2
parameter F test:         F=0.3971  , p=0.6724  , df_denom=590, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.9776  , p=0.4029  , df_denom=587, df_num=3
ssr based chi2 test:   chi2=2.9678  , p=0.3966  , df=3
likelihood ratio test: chi2=2.9604  , p=0.3978  , df=3
parameter F test:         F=0.9776  , p=0.4029  , df_denom=587, df_num=3


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


***Causality Testing for Volatility***

In [36]:
df = merged_data.copy()

In [46]:
# Assume df has columns: ['Close', 'Aggregate_Sentiment']
# Sort by date if not already
df = df.sort_index()

# Calculate daily returns (in percentage)
df['Daily_Return'] = df['Close'].pct_change() * 100

# Choose a rolling window (e.g., 10 days) for volatility
window_size = 5
df['Volatility'] = df['Daily_Return'].rolling(window=window_size).std()

# Drop the initial NaNs from rolling calculation
df = df.dropna(subset=['Volatility'])


In [47]:
df = df.dropna(subset=['sentiment_score', 'Volatility'])

In [48]:
from statsmodels.tsa.stattools import grangercausalitytests

max_lag = 3
significance_level = 0.05


In [53]:
test_data_1 = df[['Volatility', 'sentiment_score']]
results_1 = grangercausalitytests(test_data_1, maxlag=max_lag, verbose=False)

# Check if any lag shows significance
sent_cause_vol = False
for lag in range(1, max_lag+1):
    p_value = results_1[lag][0]['ssr_ftest'][1]
    if p_value < significance_level:
        sent_cause_vol = True
        print(f"At lag {lag}, p-value={p_value:.7f}, sentiment_score Granger causes Volatility.")
        break

if not sent_cause_vol:
    print("No evidence that Aggregate_Sentiment Granger causes Volatility at tested lags.")


No evidence that Aggregate_Sentiment Granger causes Volatility at tested lags.


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [50]:
test_data_2 = df[['sentiment_score', 'Volatility']]
results_2 = grangercausalitytests(test_data_2, maxlag=max_lag, verbose=False)

# Check if any lag shows significance
vol_cause_sent = False
for lag in range(1, max_lag+1):
    p_value = results_2[lag][0]['ssr_ftest'][1]
    if p_value < significance_level:
        vol_cause_sent = True
        print(f"At lag {lag}, p-value={p_value:.7f}, Volatility Granger causes sentiment_score.")
        break

if not vol_cause_sent:
    print("No evidence that Volatility Granger causes Aggregate_Sentiment at tested lags.")


At lag 1, p-value=0.0000718, Volatility Granger causes sentiment_score.


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [51]:
test_data_1 = df[['Volatility', 'Movement']]
results_1 = grangercausalitytests(test_data_1, maxlag=max_lag, verbose=False)

# Check if any lag shows significance
sent_cause_vol = False
for lag in range(1, max_lag+1):
    p_value = results_1[lag][0]['ssr_ftest'][1]
    if p_value < significance_level:
        sent_cause_vol = True
        print(f"At lag {lag}, p-value={p_value:.7f}, Movement Granger causes Volatility.")
        break

if not sent_cause_vol:
    print("No evidence that Aggregate_Sentiment Granger causes Volatility at tested lags.")


At lag 1, p-value=0.0000000, Movement Granger causes Volatility.


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [52]:
test_data_1 = df[['Movement','Volatility']]
results_1 = grangercausalitytests(test_data_1, maxlag=max_lag, verbose=False)

# Check if any lag shows significance
sent_cause_vol = False
for lag in range(1, max_lag+1):
    p_value = results_1[lag][0]['ssr_ftest'][1]
    if p_value < significance_level:
        sent_cause_vol = True
        print(f"At lag {lag}, p-value={p_value:.7f}, Volatility Granger causes Movement.")
        break

if not sent_cause_vol:
    print("No evidence that Movement Granger causes Volatility at tested lags.")


At lag 1, p-value=0.0073774, Volatility Granger causes Movement.


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


Sentiment Score Granger Causes Volatility which Granger Causes Movement. since, there is no direct causality between sentiment score and Movement, we can use Volatility as an added feature.